# Ni modificar ni ejecutar
Evaluacion en P, R, et F1 del primer modelo producido (baseline)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/"

Mounted at /content/drive


Formataje de los datos de test en containers (.spacy)
- 3 categorias generales (Per, loc, org)
- 3 categorias geograficas especificas (settlement, nature, territory)

El export de catma se realiza via el modulo de analisis, hay que utilizar la expresion siguiente `tag="%"` para generar el CSV, luego seleccionar "export as flatted CSV".

Se elimina la 1e columna y se debe agregar al CSV la primera linea con el nombre de las columnas comoe sigue, el separador es el punto y coma (;):
- document id
- document_name
- id_x
-	annotation
- context
- start_offset
- end_offset
- annotator_id
- annotator_name
- tag_name
- color
- annotation_id

In [ ]:
import pandas as pd

path_test="/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/TestDS_CATMA-Query-Result_Export-2024-01-15T12_09_12.539.csv"
df_test = pd.read_csv(path_test, sep=";")
df_test = df_test.loc[~df_test['annotator_name'].isin(['Relative'])]
df_test['tag_name'] = df_test['tag_name'].str.replace('/','_', regex=False)
#df_test.loc[df_test['annotation'].str.endswith(' '), 'end_offset'] -= 1 #no es necesario para este testDS

3 categorias P L O

In [ ]:
df_test_3plo = df_test.copy()
df_test_3plo = df_test_3plo.loc[~df_test_3plo['tag_name'].isin(['_Concept_Title-Role','_Concept_Feature Type_TerritoryFeatureType', '_Concept_Feature Type_NaturalFeatureType', '_Lemma', '_Concept_Feature Type_SettlementFeatureType',
                                  '_Concept_Social Category', '_Lemma_Additional info', '_Concept_Feature Type_Building_HumanFeatureType', '_Concept_Feature Type_MinorDistrictFeatureType',
                                  '_NonGeoEntity_Organization', '_GeoEntity', '_Error', '_Qualifier', '_Type_inherent', '_Lemma_Reference', '_Compound', '_Featuretype_last_Reference',
                                  '_Entity_lastofsametype_Reference', '_Custom_Reference', '_Entity_Lastoftype_spec' , '_Entity_last_Reference'])]

df_test_3plo['tag_name'] = df_test_3plo['tag_name'].str.replace('_NonGeoEntity_Human Group','ORG')
df_test_3plo['tag_name'] = df_test_3plo['tag_name'].str.replace('_NonGeoEntity_Person','PER')
df_test_3plo['tag_name'] = df_test_3plo['tag_name'].str.replace('_GeoEntity_Landmark','LOC')
df_test_3plo['tag_name'] = df_test_3plo['tag_name'].str.replace('_GeoEntity_Territory','LOC')
df_test_3plo['tag_name'] = df_test_3plo['tag_name'].str.replace('_GeoEntity_Settlement','LOC')

df_test_3plo_cat = df_test_3plo[['tag_name']].groupby(['tag_name'])['tag_name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
df_test_3plo_cat

,tag_name,count
0,LOC,307
2,PER,31
1,ORG,16


3 categorias geograficas (S, N, T)

In [ ]:
df_test_3cat_loc = df_test.copy()

df_test_3cat_loc = df_test_3cat_loc.loc[~df_test_3cat_loc['tag_name'].isin([ '_NonGeoEntity_Person','_NonGeoEntity_Human Group','_Concept_Title-Role','_Concept_Feature Type_TerritoryFeatureType', '_Concept_Feature Type_NaturalFeatureType', '_Lemma', '_Concept_Feature Type_SettlementFeatureType',
                                  '_Concept_Social Category', '_Lemma_Additional info', '_Concept_Feature Type_Building_HumanFeatureType', '_Concept_Feature Type_MinorDistrictFeatureType',
                                  '_NonGeoEntity_Organization', '_GeoEntity', '_Error', '_Qualifier', '_Type_inherent', '_Lemma_Reference', '_Compound', '_Featuretype_last_Reference',
                                  '_Entity_lastofsametype_Reference', '_Custom_Reference', '_Entity_Lastoftype_spec' , '_Entity_last_Reference'])]

df_test_3cat_loc_cat = df_test_3cat_loc[['tag_name']].groupby(['tag_name'])['tag_name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
df_test_3cat_loc_cat

,tag_name,count
2,_GeoEntity_Territory,116
1,_GeoEntity_Settlement,105
0,_GeoEntity_Landmark,86


CSV a JSON

In [ ]:
def transform_input_to_output(input_data):
    entities = []
    for item in input_data:
        #print(item)
        start_offset = int(item["start_offset"])
        end_offset = int(item["end_offset"])
        tag_name = item["tag_name"]
        annot_text = item["annotation"]
        if start_offset is not None and end_offset is not None and tag_name is not None:
          if(annot_text.endswith(' ')):
            entities.append((start_offset, end_offset-1, tag_name, annot_text[:-1])) #correct extra space at the end
          else:
            entities.append((start_offset, end_offset, tag_name, annot_text))
    output = {'entities': entities}
    return output

json_test_data_3plo = df_test_3plo.to_json(orient = 'records')
json_test_data_3plo_obj = list(eval(json_test_data_3plo.replace('null', 'None')))
spacy_test_data_3plo = transform_input_to_output(json_test_data_3plo_obj)
print("test ds 3plo:")
print(len(spacy_test_data_3plo['entities']))

json_test_3cat_loc = df_test_3cat_loc.to_json(orient = 'records')
json_test_3cat_loc_obj = list(eval(json_test_3cat_loc.replace('null', 'None')))
spacy_test_3cat_loc = transform_input_to_output(json_test_3cat_loc_obj)
print("test ds 3loc:")
print(len(spacy_test_3cat_loc['entities']))

test ds 3plo:
354
test ds 3loc:
307


In [ ]:
with open(gdrive_path+"textes/TestDS_sample3.txt", encoding="utf-8") as f:
  data_txt = f.read()

import re
print("number of characters: "+str(len(data_txt)))
print("number of words (very approx.): "+str(len(re.findall(r'\w+', data_txt))))

spacy_test_data_3plo['text'] = data_txt
spacy_test_data_3plo['text'] = spacy_test_data_3plo['text'].replace("\n", "\n\n")

spacy_test_3cat_loc['text'] = data_txt
spacy_test_3cat_loc['text'] = spacy_test_3cat_loc['text'].replace("\n", "\n\n")

number of characters: 22431
number of words (very approx.): 3784


LabelStudio

In [ ]:
count = 0
results = []
for ent in spacy_test_data_3plo['entities']:
  results.append({
            "id": "ann_"+str(count),
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "start": ent[0],
              "end": ent[1],
              "score": 1,
              "text": ent[3],
              "labels": [
                ent[2]
              ]
            }
          })
  count = count + 1

ann_test_3plo = [
    {
    "data": {
      "text": spacy_test_data_3plo['text']
    },
    "predictions": [
      {
        "model_version": "one",
        "score": 1,
        "result" : results
      }
    ]
  }
]

import json

json_object_ann_test_3plo = json.dumps(ann_test_3plo, indent=4)
with open(gdrive_path+"annot_data/3catplo_TestDS_sample3.json", "w") as outfile:
  outfile.write(json_object_ann_test_3plo)


In [ ]:
count = 0
results = []
for ent in spacy_test_3cat_loc['entities']:
  results.append({
            "id": "ann_"+str(count),
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "start": ent[0],
              "end": ent[1],
              "score": 1,
              "text": ent[3],
              "labels": [
                ent[2]
              ]
            }
          })
  count = count + 1

ann_test_3loc = [
    {
    "data": {
      "text": spacy_test_3cat_loc['text']
    },
    "predictions": [
      {
        "model_version": "one",
        "score": 1,
        "result" : results
      }
    ]
  }
]

import json

json_object_ann_test_3loc = json.dumps(ann_test_3loc, indent=4)
with open(gdrive_path+"annot_data/3catLoc_TestDS_sample3.json", "w") as outfile:
  outfile.write(json_object_ann_test_3loc)

Descargar los dos archivos JSON, cargarlos en LabelStudio y convertir en IOB, luego hacer remplazamientos manuales como aqui : https://colab.research.google.com/drive/1omy-tynDIf2qApKWKGQxroQW_qMIS95K#scrollTo=dKQymDMogsoF y copiar los resultados en el drive (carpeta annot_data)

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-huggingface-hub
!pip install spacy-transformers
!python -m spacy download es_dep_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires f

In [ ]:
!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/TestDS_3catplo_project-5-at-2024-01-16-17-20-01fb6f4d_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll

!python -m spacy convert "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/annot_data/TestDS_3catLoc_project-7-at-2024-01-17-14-02-dfb83a32_modif.conll" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/" -t spacy -c conll


2024-01-19 13:42:55.872861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 13:42:55.872944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 13:42:55.875116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 13:42:55.888820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 13:42:58.349534: W tensorflow/compiler/tf2

```
python -m spacy benchmark accuracy model data_path --output --code --gold-preproc --gpu-id --displacy-path --displacy-limit --per-component --spans-key
```

In [ ]:
#!python -m spacy benchmark accuracy "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es/model-best" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/TestDS_3catplo_project-5-at-2024-01-16-17-20-01fb6f4d_modif.spacy" --displacy-path "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/validation_out/testds_model_spacy_3catPLO_gpu_trf_es/"
!python -m spacy benchmark accuracy "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es_ALL/model-best" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/TestDS_3catplo_project-5-at-2024-01-16-17-20-01fb6f4d_modif.spacy" --displacy-path "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/validation_out/model_spacy_3catPLO_gpu_trf_es_ALL/"

2024-01-30 21:40:51.152701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 21:40:51.152767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 21:40:51.154319: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 21:40:51.164258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 21:40:52.436757: W tensorflow/compiler/tf2

In [ ]:
#!python -m spacy benchmark accuracy "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es/model-best" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/TestDS_3catLoc_project-7-at-2024-01-17-14-02-dfb83a32_modif.spacy" --displacy-path "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/validation_out/testds_model_spacy_3catLoc_gpu_trf_es/"
!python -m spacy benchmark accuracy "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es_drp05/model-best" "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/bin_containers/TestDS_3catLoc_project-7-at-2024-01-17-14-02-dfb83a32_modif.spacy" --displacy-path "/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/validation_out/testds_model_spacy_3catLoc_gpu_trf_es/"

2024-01-19 13:43:46.229256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 13:43:46.229336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 13:43:46.231271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 13:43:46.240093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 13:43:47.658255: W tensorflow/compiler/tf2

Ver errores con mas detalle gracias a la matriz de confusion

3 categorias P L O

In [ ]:
import spacy
nlp_ner = spacy.load("/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catPLO_gpu_trf_es/model-best")
with open("/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/textes/TestDS_sample3.txt", encoding="utf-8") as f:
  data_txt = f.read()
text_comp = data_txt.replace("\n", "\n\n")
doc = nlp_ner(text_comp)
predicted_ents = [(entity.start_char, entity.end_char, entity.label_, entity.text) for entity in doc.ents]
print("eval ds (predicted):")
len(predicted_ents)

eval ds (predicted):


354

In [ ]:
correct_ents = spacy_test_data_3plo['entities']
tp,fp,fn,tn = 0,0,0,0
for ent in predicted_ents:
  if ent not in correct_ents:
    print("False positive (mistake):", ent)
  #else:
  #  print("true positive:", ent)
for ent in correct_ents:
  if ent not in predicted_ents:
    print("False negative (missed):", ent)
  #else:
  #  print("False postive:", ent)

# false positives
fp += len([ent for ent in predicted_ents if ent not in correct_ents])
print("# false positives: "+str(fp))

# true positives
tp += len([ent for ent in predicted_ents if ent in correct_ents])
#print("# true positives: "+str(tp))

# false negatives
fn += len([ent for ent in correct_ents if ent not in predicted_ents])
print("# false negatives: "+str(fn))

# true negatives
tn += len([ent for ent in correct_ents if ent in predicted_ents])
#print("# true negatives: "+str(tn))


False positive (mistake): (1859, 1874, 'LOC', 'Joseph de Oruña')
False positive (mistake): (2081, 2085, 'LOC', 'Laxa')
False positive (mistake): (2437, 2441, 'LOC', 'Perú')
False positive (mistake): (3088, 3094, 'LOC', 'Brasil')
False positive (mistake): (3399, 3403, 'PER', 'Roca')
False positive (mistake): (3962, 4005, 'LOC', 'dis\n\n\n\n@@@Alcedo_vol_2-0363@@@\n\n\n\n#la Bahía')
False positive (mistake): (4697, 4709, 'LOC', 'Nueva España')
False positive (mistake): (5247, 5251, 'LOC', 'Perú')
False positive (mistake): (5364, 5368, 'LOC', 'Perú')
False positive (mistake): (5731, 5748, 'LOC', 'Elisabet o Isabel')
False positive (mistake): (5775, 5781, 'LOC', 'Madera')
False positive (mistake): (7153, 7159, 'LOC', 'Canada')
False positive (mistake): (7642, 7648, 'PER', 'Cartúr')
False positive (mistake): (7999, 8007, 'ORG', 'Ingleses')
False positive (mistake): (8285, 8289, 'LOC', 'Perú')
False positive (mistake): (8489, 8495, 'LOC', 'Perlas')
False positive (mistake): (8724, 8756, 'PER

3 categorias geograficas (S, N, T)

In [ ]:
import spacy
nlp_ner = spacy.load("/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/models/model_spacy_3catLoc_gpu_trf_es/model-best")
with open("/content/drive/MyDrive/Colab Notebooks/data/Alcedo/NER/textes/TestDS_sample3.txt", encoding="utf-8") as f:
  data_txt = f.read()
text_comp = data_txt.replace("\n", "\n\n")
doc = nlp_ner(text_comp)
predicted_ents = [(entity.start_char, entity.end_char, entity.label_, entity.text) for entity in doc.ents]
print("eval ds (predicted):")
len(predicted_ents)

eval ds (predicted):


301

In [ ]:
correct_ents = spacy_test_3cat_loc['entities']
tp,fp,fn,tn = 0,0,0,0
for ent in predicted_ents:
  if ent not in correct_ents:
    print("False positive (mistake):", ent)
  #else:
  #  print("true positive:", ent)
for ent in correct_ents:
  if ent not in predicted_ents:
    print("False negative (missed):", ent)
  #else:
  #  print("False postive:", ent)

# false positives
fp += len([ent for ent in predicted_ents if ent not in correct_ents])
print("# false positives: "+str(fp))

# true positives
tp += len([ent for ent in predicted_ents if ent in correct_ents])
#print("# true positives: "+str(tp))

# false negatives
fn += len([ent for ent in correct_ents if ent not in predicted_ents])
print("# false negatives: "+str(fn))

# true negatives
tn += len([ent for ent in correct_ents if ent in predicted_ents])
#print("# true negatives: "+str(tn))


False positive (mistake): (359, 360, '_GeoEntity_Landmark', 'N')
False positive (mistake): (437, 443, '_GeoEntity_Territory', 'Tarena')
False positive (mistake): (1859, 1874, '_GeoEntity_Settlement', 'Joseph de Oruña')
False positive (mistake): (2081, 2085, '_GeoEntity_Territory', 'Laxa')
False positive (mistake): (2437, 2441, '_GeoEntity_Territory', 'Perú')
False positive (mistake): (3088, 3094, '_GeoEntity_Territory', 'Brasil')
False positive (mistake): (3950, 3955, '_GeoEntity_Territory', 'Chile')
False positive (mistake): (4697, 4709, '_GeoEntity_Territory', 'Nueva España')
False positive (mistake): (5247, 5251, '_GeoEntity_Territory', 'Perú')
False positive (mistake): (5364, 5368, '_GeoEntity_Territory', 'Perú')
False positive (mistake): (5570, 5575, '_GeoEntity_Settlement', 'Ipane')
False positive (mistake): (5742, 5748, '_GeoEntity_Landmark', 'Isabel')
False positive (mistake): (5775, 5781, '_GeoEntity_Landmark', 'Madera')
False positive (mistake): (7153, 7159, '_GeoEntity_Terri